# Training CNN on DLVM (GPU)
In this notebook we will go through the steps of stacking layers together and seeing how it affects performance. In section we went through the steps of how the various types of layers and their properties affect the dimensions of the data passing through them. In this notebook we will look at the affect on performance so that we get an idea of stacking these layers can give us better performance. We will be basic this CNN on the VGG architecture.

In [ ]:
import numpy as np
import os
import sys
import tarfile
import pickle
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import keras as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.datasets import cifar10

from urllib.request import urlretrieve

In [ ]:
# Parameters
EPOCHS = 30
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9
N_CLASSES = 10 # There are 10 classes in the CIFAR10 dataset
GPU = True

In [ ]:
def prepare_cifar(x_train, y_train, x_test, y_test):
    
    # Scale pixel intensity
    x_train = x_train / 255.0
    x_test = x_test / 255.0
        
    enc = OneHotEncoder(categorical_features='all')
    fit = enc.fit(y_train)
    y_train = fit.transform(y_train).toarray()
    y_test = fit.transform(y_test).toarray()
    
    return (x_train.astype(np.float32), 
            y_train.astype(np.int32), 
            x_test.astype(np.float32), 
            y_test.astype(np.int32))

In [ ]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [ ]:
%%time
# Data into format for library
# x_train, y_train, x_test, y_test = prepare_cifar(*load_cifar(), channel_first=False)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, y_train, x_test, y_test = prepare_cifar(x_train, y_train, x_test, y_test)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

## Model 1
Our first model will have 2 convolution layers and a max pooling layer. The classification layer will use softmax as we want it to only output a 1 for our specified class and 0 everywhere else.

In [ ]:
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(32, 32, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

model.add(Flatten())
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(N_CLASSES, activation='softmax'))

In [ ]:
%%time
model = init_model(model) # Initialise model
model.summary()

In [ ]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=20,
          verbose=1)

In [ ]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Our model gets an accuracy of 72.8% on the test set after 20 epochs. We can also see that it achieves 100% on the training set a few epochs before we stop training. It would usually be prudent to stop the model earlier and there are usually callbacks that can be used in any of the frameworks to do this. We are simply not using these here to try and keep things simple

## Model 2 
With the second model we will add a second convolution block. In keeping with the VGG architecture we will add two convolution layers each with 128 filters as well as a mac pooling layer. This time we will train it for 30 epochs.

In [ ]:
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(32, 32, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

model.add(Flatten())
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(N_CLASSES, activation='softmax'))

In [ ]:
%%time
model = init_model(model) # Initialise model
model.summary()

In [ ]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

In [ ]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

The model does slightly better with an accuracy of 75.5.

## Model 3 
For our third model we will add a 3rd convolution block. This will be made up of 3 convolution layers each with 256 filters each. Again we will have a max pooling block at the end.

In [ ]:
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(32, 32, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))


model.add(Flatten())
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(N_CLASSES, activation='softmax'))

In [ ]:
%%time
model = init_model(model) # Initialise model
model.summary()

In [ ]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

In [ ]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

The model reaches a accuracy of 76%. As you can see with each additional layer we get better results but the returns diminish with each succesive block.

## Model 4
Due to the large number of free parameters CNNs can benefit from regularisation. One way to refularise is to use a dropout layer which we talked about earlier. This layer will randomly during the forward pass zero a certain proportion of its outputs. This was also eployed by the authors of the VGG architecture.

In [ ]:
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(32, 32, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

 # Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

model.add(Flatten())
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dropout(0.5))
model.add(Dense(N_CLASSES, activation='softmax'))

In [ ]:
%%time
model = init_model(model) # Initialise model
model.summary()

In [ ]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

In [ ]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

We can see that our accuracy has increased further to 80%. The VGG architecture actually has even more layers than our final model but it was designed to tackle the ImageNet dataset which contains a lot more data than the CIFAR10 dataset. Adding further layers with the limited data available would quickly prove untenable. 